## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint
from pylab import savefig

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kirakira,SB,-10.4544,161.9205,82.54,79,39,4.65,scattered clouds
1,1,Coolum Beach,AU,-26.5333,153.1000,86.34,40,0,5.86,clear sky
2,2,Barrow,US,71.2906,-156.7887,28.29,84,100,14.09,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,51.91,67,42,2.51,scattered clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,75,4.61,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hermanus,ZA,-34.4187,19.2345,51.91,67,42,2.51,scattered clouds
7,7,Cooma,AU,-36.2333,149.1333,62.37,55,75,12.66,broken clouds
8,8,San Juan,AR,-31.5375,-68.5364,59.72,19,63,15.99,broken clouds
11,11,Cape Town,ZA,-33.9258,18.4232,52.12,71,40,2.30,scattered clouds
12,12,Oranjemund,NaN,-28.5500,16.4333,52.65,75,0,19.86,clear sky
20,20,Rio Grande,BR,-32.0350,-52.0986,57.94,88,68,9.86,broken clouds
22,22,Mataura,NZ,-46.1927,168.8643,55.47,63,100,1.86,overcast clouds
24,24,Tumen,CN,42.9661,129.8425,58.35,79,100,2.82,overcast clouds
26,26,Busselton,AU,-33.6500,115.3333,57.09,47,29,14.07,scattered clouds
29,29,Campbell River,CA,50.0163,-125.2446,54.93,96,90,1.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                184
City                   184
Country                182
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                182
City                   182
Country                182
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hermanus,ZA,51.91,scattered clouds,-34.4187,19.2345,
7,Cooma,AU,62.37,broken clouds,-36.2333,149.1333,
8,San Juan,AR,59.72,broken clouds,-31.5375,-68.5364,
11,Cape Town,ZA,52.12,scattered clouds,-33.9258,18.4232,
20,Rio Grande,BR,57.94,broken clouds,-32.0350,-52.0986,
22,Mataura,NZ,55.47,overcast clouds,-46.1927,168.8643,
24,Tumen,CN,58.35,overcast clouds,42.9661,129.8425,
26,Busselton,AU,57.09,scattered clouds,-33.6500,115.3333,
29,Campbell River,CA,54.93,overcast clouds,50.0163,-125.2446,
38,Albany,US,62.31,scattered clouds,42.6001,-73.9662,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
       
             

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
pprint.pprint(hotels)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -8.2228234,
                                        'lng': -78.97657009999999},
                           'viewport': {'northeast': {'lat': -8.221476169708497,
                                                      'lng': -78.97523246970849},
                                        'southwest': {'lat': -8.224174130291502,
                                                      'lng': -78.9779304302915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'ELACORP Alojamiento',
              'place_id': 'ChIJ0_SO7kUZrZERPqWXdbVVW4Y',
              'plus_code': {'compound_code': 'Q2GF+V9 Salaverry, Peru',
                            'global_code': 

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.count()

City                   182
Country                182
Max Temp               182
Current Description    182
Lat                    182
Lng                    182
Hotel Name             182
dtype: int64

In [11]:
#Review that CSV file has proper format
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hermanus,ZA,51.91,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Cooma,AU,62.37,broken clouds,-36.2333,149.1333,High Country Motel
8,San Juan,AR,59.72,broken clouds,-31.5375,-68.5364,Hotel Provincial
11,Cape Town,ZA,52.12,scattered clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
20,Rio Grande,BR,57.94,broken clouds,-32.0350,-52.0986,Hotel Atlântico Rio Grande


In [12]:
# 8a. Create the output File (CSV)
clean_hotel_df=hotel_df.copy()
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Save figure ( a manual screen shot)
#savefig("../Vacation_Search/WeatherPy_vacation_map.png")

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))